In [1]:
print("COUCOU")

COUCOU


In [2]:
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

from matplotlib import pyplot as plt
import IPythonMagic
from Timer import Timer


In [3]:
%setup_logging

Python version 3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 09:53:17) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [4]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (8, 0, 0)
Driver version 9010
Using 'Quadro K2000' GPU
 => compute capability: (3, 0)
 => memory: 1906 / 1999 MB available
Created context handle <34145840>
Using CUDA cache dir /home/jobic/test/cuda/milan2018/MilanoGPU2018/notebooks/cuda_cache


In [6]:
kernel_src="""
__global__ void shmemReduction(float* output, float *input, int size) {
    //First we stride through global memory and compute
    //the maximum for every thread
    int gid = blockIdx.x * blockDim.x + threadIdx.x; //threadIdx.x == 0 because we use just one block
    
    float max_value = -9999999; //FIXME : Use proprer value here
    for (int i = threadIdx.x; i < size; i += blockDim.x) {
        max_value = fmaxf(max_value,input[i]);
    }
    
    //Temporary write to memory to check if things work so far
    output[threadIdx.x] = max_value;
    
    //Store the per-thread maximum in shared memory
    volatile __shared__ float max_shared[128];
    max_shared[threadIdx.x] = max_value;
    
    //synchronize so that all thread see the same shared memory
    __syncthreads();
    
    // Find the maximum in shared memory
    // Reduce from 128 to 64 elements
    if (threadIdx.x < 64) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 64]);
    }
    //Since we here have monre than one active wrap (threadIdx.x>64)
    //we want to make sure everyone finished before continuing
    __syncthreads();    
    
    // Reduce from 64 to 32 elements
    if (threadIdx.x < 32) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 32]);
    }
    if (threadIdx.x < 16) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 16]);
    }
    // Reduce from 16 to 8 elements
    if (threadIdx.x < 8) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 8]);
    }
    // Reduce from 8 to 4 elements
    if (threadIdx.x < 4) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 4]);
    }
    // Reduce from 4 to 2 elements
    if (threadIdx.x < 2) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 2]);
    }    
    // Reduce from 2 to 1 elements
    if (threadIdx.x < 1) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x + 1]);
    }    
    
    //finally write out the output
    if (threadIdx.x == 0) {
        output[0] = max_shared[0];
    }
}
"""
kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction");

/home/jobic/anaconda3/envs/gpudev/lib/python3.6/site-packages/ipykernel/__main__.py:61: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(6): warning: variable "gid" was declared but never referenced

kernel.cu(6): warning: variable "gid" was declared but never referenced




In [11]:
n = 512
a = np.random.random((1,n)).astype(np.float32)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 16
b = np.empty((1,num_threads)).astype(np.float32)
b_g = GPUArray(b.shape, b.dtype)


In [12]:
grid_size = (num_threads, 1, 1)
block_size = (1, 1, 1)

kernel_function(b_g,a_g,np.int32(n),grid=grid_size,block=block_size)

b_g.get(b)

#print(a)
print(b)
print(np.max(a))

[[0.9967803       nan       nan       nan       nan       nan       nan
        nan       nan       nan       nan       nan       nan       nan
        nan       nan]]
0.9967803
